In [1]:
import json
import math
import requests

from pandas import datetime, Series
from urllib.request import urlopen
from tkinter import *
from urllib.parse import urlencode
import os
import pandas as pd


In [2]:
dir = os.listdir('./data')
dir.sort()
xlsx_list = [file for file in dir if file.startswith('Daily')]

In [3]:
dir2 = os.listdir('./data')
dir2.sort()
xlsx_list2 = [ file for file in dir2 if file.startswith('search')]

In [4]:
for i, data in enumerate(xlsx_list):
    if i == 0:
        df = pd.read_excel('./data/'+data)
    else:
        small_df = pd.read_excel('./data/'+data)
        df = pd.concat([df, small_df]).reset_index(drop = True)

In [5]:
for i, data in enumerate(xlsx_list2):
    if i == 0:
        df2 = pd.read_excel('./data/'+data)
    else:
        small_df2 = pd.read_excel('./data/'+data)
        df2 = pd.concat([df2, small_df2]).reset_index(drop = True)

# 2008~2017 일일 박스오피스 데이터 셋



In [6]:
df

,movieNm,movieCd,openDt,rank,scrnCnt,salesAmt,salesShare,audiCnt,showCnt
0,황금 나침반,20070551,2007-12-18,1,293,1062569500,18.8,161052,1682
1,내셔널 트레져:비밀의 책,20070549,2007-12-19,2,246,827405500,14.7,123779,1168
2,기다리다 미쳐,20070661,2008-01-01,3,238,497751500,8.8,73572,1553
3,색즉시공 시즌2,20070637,2007-12-12,4,208,428237000,7.6,61740,811
4,마고리엄의 장난감 백화점,20070586,2007-12-24,5,191,352225000,6.2,55387,856
5,P.S. 아이 러브 유,20071003,2008-01-03,6,184,349718000,6.2,49842,1061
6,나는 전설이다,20070550,2007-12-12,7,186,325831000,5.8,48077,749
7,앨빈과 슈퍼밴드,20070562,2007-12-19,8,157,304514500,5.4,47718,607
8,어거스트 러쉬,20070585,2007-11-29,9,124,320816500,5.7,47398,476
9,아메리칸 갱스터,20070642,2007-12-27,10,198,291855500,5.2,41130,626


In [7]:
def make_movie_df(day_list):
    movie_df = pd.DataFrame(columns = ["movieNm", 'movieCd',  "openDt", "rank", "scrnCnt", 
                                       "salesAmt", 'salesShare', 'audiCnt','showCnt'])
    for i in day_list:
        try:
            for data in get_movie_data(i)['boxOfficeResult']['dailyBoxOfficeList']:
                
                movie_df.loc[len(movie_df)] = [
                    data["movieNm"],
                    data['movieCd'],
                    data["openDt"],
                    data["rank"],
                    data["scrnCnt"],
                    data['salesAmt'],
                    data['salesShare'],
                    data['audiCnt'],
                    data['showCnt']
                    
                

                 
                ]
        except:
            i
    return movie_df

In [8]:
df2

,searchDay
0,20080101
1,20080101
2,20080101
3,20080101
4,20080101
5,20080101
6,20080101
7,20080101
8,20080101
9,20080101


In [19]:
DailyData_set = pd.merge(df, df2,left_index=True, right_index=True, how='left')

In [21]:
DailyData_set.to_excel('DailyData_set(2008-2017).xlsx', encoding='utf-8')
